# Machine Learning with scikit-learn

In the lessons so far, we have mostly just used the default `.score()` method of fitted models.  For most or all classification models, this measures accuracy.  For most or all regression models, this measures $R^2$ (coefficient of determination).  As we have mentioned, the subpackage `sklearn.metrics` contains a large number of other scorers.  Depending on your purpose, one of these might be more appropriate.

In this lesson we will look at a few such metrics, but we will also develop a custom metric that is not included in scikit-learn (and presumably never will be, for reasons we will see).

In [1]:
%matplotlib inline
from src.setup import *

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## A slightly unbalanced classification

For this lesson, we will look at the [Dermatology Data Set](https://archive.ics.uci.edu/ml/datasets/Dermatology) available from UCI.  This data contains 34 measurements of 366 patients, with each one diagnosed as having one of six skin conditions.  Our purpose in using this data is two-fold. On the one hand, we want to look at a multi-class classification problem, which we have not done extensively in these lessons.  But more interestingly at the end, we want to look at the value of non-top diagnoses, which may have utility for particular domain problems.

> <small>Dua, D. and Graff, C. (2019). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.</small>

### Digression on multi-labels

Note that what we present here is **not** a multi-label problem.  In some situations it is useful to identify more than one class to which a sample might belong.  In the current domain, that would be patients who have multiple skin conditions at once.  Such is possible, but this dataset is assumed not to contain that situation.  Or in another domain, we might wish to characterize a photographic image by multiple classes.  For example, an image containing both a cat and a dog would get both of these labels, but would get none of the, e.g. other 98 available labels because those things were not in the image.  Multi-label problems can be addressed with scikit-learn.

See the official documentation of [multiclass and multilabel algorithms](https://scikit-learn.org/stable/modules/multiclass.html).  Note that multi-output is related to multi-label, but is a somewhat different concept.  In multi-label, any number of labels may be identified (including zero).  This is akin to one-hot encoding, but of the output (maybe "multi-hot" would be a good description).  

In contrast, mutli-output identifies a fixed number of outputs, which we might think of as orthogonal dimensions of the output.  In a sense this is like the fixed number of input features.  For example, in the photo classification problem, we might always want to predict `(color, subject, lighting)` for every image.  So sometimes it is a "brown dog in daylight", other times it is a "white cat at night."

Basically all classification models can be transformed into multi-label algorithms by transforming the problem into a collection of one-vs-all classifiers.  For example, one model is cat-vs-not-cat; another model is dog-vs-not-dog.  Similarly for all of the stipulated 100 known classes.  If both the cat-vs-not-cat and dog-vs-not-dog models make a positive prediction, we would assign both those labels.  However, other models are inherently multi-label by their design, so this kind of transformation is irrelevant (and counter-productive, in fact) if you use those.

### The dataset

We get this data in somewhat raw form.  The `dermatology.data` file is a CSV with no headers.  The `dermatology.names` files contains a bit more than its name might suggest.  Beyond providing the feature names, it gives additional exposition of the dataset, such as value coding, where unknown values occur, and a few other things in prose.  I produced a code-friendly extraction of the relevant information below.

In [2]:
# Histopathological Attributes: (values 0, 1, 2, 3)
# Clinical Attributes: (values 0, 1, 2, 3, unless indicated)
features = [
    "erythema",
    "scaling",
    "definite borders",
    "itching",
    "koebner phenomenon",
    "polygonal papules",
    "follicular papules",
    "oral mucosal involvement",
    "knee and elbow involvement",
    "scalp involvement",
    "family history",  # 0 or 1
    "melanin incontinence",
    "eosinophils in the infiltrate",
    "PNL infiltrate",
    "fibrosis of the papillary dermis",
    "exocytosis",
    "acanthosis",
    "hyperkeratosis",
    "parakeratosis",
    "clubbing of the rete ridges",
    "elongation of the rete ridges",
    "thinning of the suprapapillary epidermis",
    "spongiform pustule",
    "munro microabcess",
    "focal hypergranulosis",
    "disappearance of the granular layer",
    "vacuolisation and damage of basal layer",
    "spongiosis",
    "saw-tooth appearance of retes",
    "follicular horn plug",
    "perifollicular parakeratosis",
    "inflammatory monoluclear inflitrate",
    "band-like infiltrate",
    "Age",  # linear; missing marked '?'
    "TARGET"  # See mapping
]

For reference and later use, the dictionary `targets` contains the class code and name of the skin condition diagnosed.  We also note here the number of observations of each condition.  They are somewhat imbalanced, which might affect the metrics we use.  That is, in this dataset, psorisis is much more common than pilaris.  I am not a dermatologist, and have no idea what the prevalence of these conditions is in the general population; there may have been selection bias in this aggregation.  That is, beyond the obvious selection bias that people with no skin conditions at all are not included.

In [3]:
targets = {
    1:"psoriasis",                 # 112 instances
    2:"seboreic dermatitis",       # 61
    3:"lichen planus",             # 72
    4:"pityriasis rosea",          # 49
    5:"cronic dermatitis",         # 52    
    6:"pityriasis rubra pilaris",  # 20
}

Reading in the data needs minor massaging to be ready for use.  To have a friendly DataFrame to work with, we attach the names of the features as columns.  But the missing `Age` that is marked with a questino mark needs extra clean-up.  I have decided to impute the median age for missing data. Other approaches are possible, and some models will work with missing data.  As my domain judgement, I chose this approach.

In [4]:
df = pd.read_csv('data/dermatology.data', header=None, names=features)
df.loc[df.Age == '?', 'Age'] = None
df['Age'] = df.Age.astype(float)
df.loc[df.Age.isnull(), 'Age'] = df.Age.median()
df

,erythema,scaling,definite borders,itching,koebner phenomenon,polygonal papules,follicular papules,oral mucosal involvement,knee and elbow involvement,scalp involvement,...,disappearance of the granular layer,vacuolisation and damage of basal layer,spongiosis,saw-tooth appearance of retes,follicular horn plug,perifollicular parakeratosis,inflammatory monoluclear inflitrate,band-like infiltrate,Age,TARGET
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55.0,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8.0,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26.0,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40.0,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2,1,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,2,0,25.0,4
362,3,2,1,0,1,0,0,0,0,0,...,1,0,1,0,0,0,2,0,36.0,4
363,3,2,2,2,3,2,0,2,0,0,...,0,3,0,3,0,0,2,3,28.0,3
364,2,1,3,1,2,3,0,2,0,0,...,0,2,0,1,0,0,2,3,50.0,3


## Training a model

The usual steps can be done here.  We create our X and y arrays for the features and target.  We perform a train/test split on the data.  For this problem, we will use a k-nearest neighbors model.  I have not tried a wide variety of models or hyperparameters, and have no idea what the "best" model is.  But KNN is often quite good, and it is a good way to illustrate the concepts here.

In [27]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [28]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

0.9239130434782609

Let us also create a collection of predictions against the test data that we will utilize below.  For convenience, we can transform the array result into a Pandas Series so that the index matches between the ground truth of the training data and the predictions.

In [29]:
y_pred = pd.Series(knn.predict(X_test), index=y_test.index)
y_pred.map(targets)

247              psoriasis
127          lichen planus
230    seboreic dermatitis
162          lichen planus
159    seboreic dermatitis
              ...         
321       pityriasis rosea
59     seboreic dermatitis
12     seboreic dermatitis
312          lichen planus
107              psoriasis
Length: 92, dtype: object

## Evaluating the trained model

So far, so good.  The usual fit-then-score steps work as expected.  But in particular, we simply used the default `.score()` method attached to the trained model object.  For this classification, that default is *accuracy*.  We saw earlier, in the introductory material, that depending on our purpose, accuracy might not be the most useful metric.  

The decision of a metric is very much driven by our "business requirement" and there is not single objective answer.  However, one thing that is absolute is that when we get to comparing different models to each other—whether entirely different styles of models, or different hyperparameters—we need some way of quantifying the **goodness** of a model to choose which one to keep.  In particular, in practical terms we need to reduce the "goodness" to a single number we can compare among modeling approaches.

### Precision, recall, and f1 score

How good is our fitted model by some other metrics we dicussed in the very first ["What Is Machine Learning?"](WhatIsML.ipynb) lesson?  Perhaps you should review that lesson for the following discussion.  One matter is that the default "averaging" technique assumes a binary classification.  For our multi-class model we have to chose something different.  There are three options here:

* `'micro'`: Count false positives, false negatives, true positives, true negatives for all observations independently, and simply perform row-wise aggregation of the counts.
* `'macro'`: Count the true and false categories grouping by class label, and take the mean of all those scores per label.
* `'weighted'`: Similar to macro, but take a weighted average based on the "support" (frequency) of each different label.

Again, there is no uniformly right answer to which of these is the best.  It depends on your requirements.

In [30]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [31]:
precision_score(y_pred, y_test, average='micro')

0.9239130434782609

In [32]:
f1_score(y_pred, y_test, average='weighted')

0.9240042566129523

In [33]:
recall_score(y_pred, y_test, average='macro')

0.87699709513435

These numbers are not especially far apart, but the different metrics absolutely give us different results.  Which one we choose will give different answers for which model we should choose for the production system.  

Let us create a different model and compare it to the first one under different metrics.  A confession here is that I easily identified a number of model types and hyperparameters that are clearly better than the first one under almost any metric.  The `knn` and `knn2` objects are simply "naive" attempts that show the pattern I waish to demonstrate.  That is, among these two models, chosing one is sensitive to which metric you prefer.  But for those better models I found, the same general pattern will emerge, just with higher numbers on each metric.

In [34]:
knn2 = KNeighborsClassifier(n_neighbors=2, metric="manhattan")
knn2.fit(X_train, y_train)
y_pred2 = knn2.predict(X_test)

In [35]:
print('Model 1:', precision_score(y_pred, y_test, average='micro'))
print('Model 2:', precision_score(y_pred2, y_test, average='micro'))   

Model 1: 0.9239130434782609
Model 2: 0.8369565217391305


In [36]:
print('Model 1:', f1_score(y_pred, y_test, average='weighted'))
print('Model 2:', f1_score(y_pred2, y_test, average='weighted'))

Model 1: 0.9240042566129523
Model 2: 0.8411545022822631


In [37]:
print('Model 1:', recall_score(y_pred, y_test, average='macro'))
print('Model 2:', recall_score(y_pred2, y_test, average='macro'))

Model 1: 0.87699709513435
Model 2: 0.8525925925925927


### Revisiting accuracy

The most notable limitation of using accuracy as a metric is its poor handling of unbalanced classes.  That might be too stark a way to phrase it though.  For *some purposes* class distribution matters. As with all metrics, our choices are driven by task requirements.  However, let us quickly review the problem case we presented in the first lesson:

Consider a test/model for an uncommon but serious disease:

| Predict/Actual | Positive | Negative |
|----------------|----------|----------|
| Positive       |    1     |    0     |
| Negative       |    2     |   997    | 


This has an accuracy of 99.8% (997 true negatives, 1 true positive).  But it misses the requirement that detecting the true positives is, by far, the most important goal.  This is a situation where *balanced accuracy* is likely to be more relevant.

In [38]:
from sklearn.metrics import balanced_accuracy_score as ba, accuracy_score

Let us construct these hypothetical predictions quickly, simply as results.  For ease of creation, we simply put the True values at the start of each array.

In [39]:
disease_actual = pd.Series([1, 1, 1] + [0]*997)
disease_predict = pd.Series([1] + [0]*999)
accuracy_score(disease_predict, disease_actual)

0.998

We have to be careful about balanced accuracy because there is a question of "balanced on what?"  Notice this difference:

In [40]:
print(f"Balanced on prediction: "
      f"{ba(disease_predict, disease_actual):.3f}")
print(f"Balanced on actual: "
      f"{ba(disease_actual, disease_predict):.3f}")

Balanced on prediction: 0.999
Balanced on actual: 0.667


In other words, since predictions are almost always of "negative", the balance weighted by prediction pays even more attention to the negative cases.  This is almost surely not what we want.  Rather, we would like to overweight the rare case; or more specifically, the "positive" case.

We can achieve more fine-tuned control of what we care about by explicitly setting sample weights.  We need not even set the same weight for each prediction of the same class as balanced accuracy does.  It *could be* that we find the result for samples 1, 5, 10, 35, and 72 far more important regardless of which prediction they make, for example.  This is likely an edge case, but it is available.

To replicate what balanced accuracy does, we can map weights (per sample/row) to the *reverse* of the actual counts.  Here the order of the actual/prediction arguments does not matter.

In [41]:
accuracy_score(disease_actual, disease_predict, 
               sample_weight=disease_actual.map({0:3, 1:997}))

0.6666666666666666

Note that the number we get here with weighted accuracy is not precisely the same as any of accuracy, f1 score, precision, or recall.  Perhaps we know, moreover, for this task requirement that positive cases are even more important than their distribution suggests. We can easily use that as our metric.

In [44]:
accuracy_score(disease_predict, disease_actual, 
   sample_weight=disease_actual.map({0:1, 1:1_000_000}))

0.33355481528305425

### Back to multi-class

Having more starkly illustrated balance issues with a highly-imbalanced binary classification problem, let us return to the multi-class skin condition model.  We *could* use balanced accuracy as a higher level function, but instead we will jump directly to using explicit sample weights.  This, in principle, allows us to make more nuanced evaluation.

However, it is not obvious in the abstract—to repeat the point yet again (that you are perhaps sick of)—what the best balance or weights are.  Is it more important to diagnose the common conditions correctly, or the rare conditions? Is there one condition in particular that it is more important to get right (maybe it is more serious, or requires a more specific treatment)?  You simply have to know the problem domain to make these decisions.

As a start, we probably want a sense of the distribution of the conditions.  That was given in the metadata descriptions above, but we will repeat it in code.  Moroever, we presumably want the distribution within the overall dataset, not only within the test set.  Using `balanced_accuracy_score()` would not give us the opportunity to look at the wider distribution.  In fact, quite possibly we could know the general distribution of these conditions beyond the 366 training data samples we have, and utilize that information.

In [45]:
counts = y.value_counts()
counts.index = counts.index.map(targets)
counts

psoriasis                   112
lichen planus                72
seboreic dermatitis          61
cronic dermatitis            52
pityriasis rosea             49
pityriasis rubra pilaris     20
Name: TARGET, dtype: int64

These counts will allow us to overweight the common classes, but as we have seen, quite often this is exactly the inverse of what we want.  Fortunately, it is easy to construct that inverse.  The scaling of these numbers is not important, simply their ratios.

In [46]:
weights = 1/counts
weights

psoriasis                   0.008929
lichen planus               0.013889
seboreic dermatitis         0.016393
cronic dermatitis           0.019231
pityriasis rosea            0.020408
pityriasis rubra pilaris    0.050000
Name: TARGET, dtype: float64

In [47]:
accuracy_score(y_pred, y_test)

0.9239130434782609

For convenience of presentation, we create a mapping from the index of the test set to the condition names.

In [48]:
y_names = y_test.map(targets)
y_names

247              psoriasis
127          lichen planus
230    seboreic dermatitis
162          lichen planus
159    seboreic dermatitis
              ...         
321       pityriasis rosea
59     seboreic dermatitis
12     seboreic dermatitis
312          lichen planus
107              psoriasis
Name: TARGET, Length: 92, dtype: object

It is common that overweighting the frequent target classes produces increased accuracy.  After all, a DummyClassifier can simply predict the most common class with no effort.  Actual models also have a tendency to overpredict common labels; not always, but often enough to be a concern.

In [49]:
accuracy_score(y_pred, y_test, sample_weight=y_names.map(counts))

0.9382104342757214

The inverse weighting produces a different answer, usually lower than the naive accuracy.  

In [50]:
accuracy_score(y_pred, y_test, sample_weight=y_names.map(weights))

0.9114370783796502

Clearly, your ultimate goal will not simply be to compare different metrics, but to compare different models once you have determined the appropriate metric.

## A custom metric

The standard metrics in scikit-learn or other machine learning libraries have a lot of uses, and are greatly configurable.  However, for some particular task requirement, you may need a different functional form.  

The metric we construct here is closely inspired by an job I worked at.  That was a company that made consumer recommendations.  The basic idea is that sometimes predictions other than the top one are still relevant.  In the context of this dermatological diagnosis problem, we can imagine that it is likewise useful to have a second most likely diagnosis made available to physicians and counting towards the goodness of the model.  For example, perhaps treatments would be considered that would apply to both diagnoses, or an easier transition to a second treatment would be available with the secondary possible diagnosis.

Happily, as we have seen before, many types of models can provide not only a single prediction but also probabilities among all the labels.  We get that with the KNN model we used.

In [51]:
probs = pd.DataFrame(knn.predict_proba(X_test), columns=targets.values())
probs.sample(10)

,psoriasis,seboreic dermatitis,lichen planus,pityriasis rosea,cronic dermatitis,pityriasis rubra pilaris
39,0.0,0.4,0.0,0.6,0.0,0.0
42,0.0,0.0,1.0,0.0,0.0,0.0
79,0.0,0.0,1.0,0.0,0.0,0.0
87,0.0,0.4,0.0,0.6,0.0,0.0
33,1.0,0.0,0.0,0.0,0.0,0.0
35,1.0,0.0,0.0,0.0,0.0,0.0
70,0.0,0.4,0.0,0.0,0.6,0.0
63,0.0,0.0,0.0,0.0,1.0,0.0
20,0.0,0.2,0.0,0.6,0.2,0.0
90,0.0,0.0,1.0,0.0,0.0,0.0


In some cases, a single diagnosis is assigned a 100% probability.  If correct, that should be worth a lot.  If it is entirely wrong, the model should be penalized.  However, in other samples the evaluation is divided between multiple diagnoses, with various probabilities strictly between 0% and 100%.  We would like to give "partial credit" to those rows.

The function below is implemented for clarity, not for speed.  Specifically, we would figure out how to vectorize functions that operate on NumPy array or Pandas DataFrames or Series if we had a large dataset.  Here we loop through the probabilities and targets to make the code easier to understand.  The algorithm we want here is to give, for each row, a score:

* 1.0 if the highest probability is the correct label
* 0.8 if two or more labels have equal probability and one is correct
* 0.5 if the second highest probability is the correct label
* 0.0 otherwise

The final score is the mean of all the row scores.

In [54]:
def first_or_second(estimator, X, y):
    assert len(X) == len(y)
    
    # Get probabilities
    probs = estimator.predict_proba(X)
    
    # Empty array to hold the scores
    scores = np.empty(len(y), dtype=float)

    # ... map labels to position in the probs array
    lab2ndx = {label: ndx 
               for ndx, label in enumerate(estimator.classes_)}
    # y might start as either Pandas Series or NumPy array
    target = pd.Series(y).map(lab2ndx).to_numpy()

    for i in range(len(target)):
        prob = probs[i]
        best_val = prob.max()    # highest prob
        best_ats = np.where(prob == best_val)[0]
        # Only one 2nd place as implemented
        the_rest = np.where(prob != best_val)[0]
        best_of_rest = np.argmax(the_rest)

        if len(best_ats) == 1:  # just one best prediction
            # ... and it is correct
                if best_ats[0] == target[i]:
                    scores[i] = 1.0
                # ... or 2nd place is correct
                elif best_of_rest == target[i]:
                    scores[i] = 0.5
                # ... neither 1st nor second is right
                else:
                    scores[i] = 0.0
        else:                   # tie for best prediction
            scores[i] = 0.0     # start pessimistic
            # if any winners are right, boost score
            for ndx in best_ats:
                if ndx == target[i]:
                    scores[i] = 0.8
                    break

    return scores.mean()

The concrete result of this metric is slightly higher than others we have looked at, which you would expect since we give credit for "second recommendation."  All that actually matters is that it has the property that higher scores are better.  We can use any numeric range we want for scores, but this metric also falls between 0 and 1 like most others do.  Since we are looking at more than just a single prediction per row, we need to pass in the actual estimator (model) we want to use.

In [55]:
first_or_second(knn, X, y)

0.9286885245901638

In [56]:
first_or_second(knn2, X, y)

0.9721311475409836

### The scorer API

We have created a useful enough function that can be used manually.  But it is also compliant with the *scorer API* and hence can be used in conjunction with other kinds of validation functions.  For example, we might want to use our custom scorer with cross-validation.

In [57]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

Remember the plain case of cross-validation, which gives us several scores, one for each slice, We could average them or do other operations:

In [58]:
cross_val_score(knn, X, y, cv=5)

array([0.87837838, 0.79452055, 0.8630137 , 0.89041096, 0.89041096])

Another option is to generate a scorer from a metric using the `make_scorer()` utility function.

In [59]:
cross_val_score(knn, X, y, cv=5,
                scoring=make_scorer(f1_score, average="macro")) 

array([0.85635755, 0.77090356, 0.84213291, 0.85470189, 0.86390039])

However, of interest to us is that we can likewise use the custom `first_or_second()` scorer function.

In [60]:
cross_val_score(knn, X, y, scoring=first_or_second, cv=5)

array([0.88108108, 0.82739726, 0.90684932, 0.90821918, 0.89863014])

In [61]:
cross_val_score(knn2, X, y, scoring=first_or_second, cv=5)

array([0.92972973, 0.95616438, 0.95890411, 0.93972603, 0.93424658])